<a href="https://colab.research.google.com/github/victorluongo/fiap-techchallenge-fase2/blob/main/tech_challenge_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema da Mochila com Algoritmo Genético

## Introdução

O problema da mochila é um clássico em otimização, onde o objetivo é maximizar o valor total de itens que podem ser colocados em uma mochila, respeitando um limite de peso. Neste notebook, vamos explorar uma implementação do algoritmo genético para resolver esse problema.

## Estrutura do Código

O código é dividido em várias partes, cada uma com uma função específica. Vamos analisar cada uma delas.

### 1. Importação de Bibliotecas

In [ ]:
import random
import numpy as np
import sys

Aqui, importamos as bibliotecas necessárias `random` e `numpy` são utilizados para operações aleatórias e numéricas.

### 2.1. Definição de Parâmetros - P3

In [ ]:
pesos = [56,59,80,64,75,17]
valores = [50, 50, 64, 46, 50, 5]
capacidade_maxima = 190
N_ITENS = len(pesos)

tamanho_populacao = 20
taxa_mutacao = 0.2
numero_geracoes = 1000

Aqui, definimos os pesos e valores dos itens, a capacidade máxima da mochila e o número total de itens.

### 2.2. Definição de Parâmetros - P8

In [ ]:
pesos = [382745,799601,909247,729069,467902,44328,34610,698150,823460,903959,853665,551830,610856,670702,488960,951111,323046,446298,931161,31385,496951,264724,224916,169684]
valores = [825594,1677009,1676628,1523970,943972,97426,69666,1296457,1679693,1902996,1844992,1049289,1252836,1319836,953277,2067538,675367,853655,1826027,65731,901489,577243,466257,369261]
capacidade_maxima = 6404180
N_ITENS = len(pesos)

tamanho_populacao = 20
taxa_mutacao = 0.2
numero_geracoes = 1000

Aqui, definimos os pesos e valores dos itens, a capacidade máxima da mochila e o número total de itens.

### 3. Funções do Algoritmo Genético

#### Função de Aptidão

In [ ]:
def funcao_aptidao(individuo):
  peso_total = sum([pesos[i] for i in range(len(individuo)) if individuo[i] == 1])
  valor_total = sum([valores[i] for i in range(len(individuo)) if individuo[i] == 1])
  if peso_total > capacidade_maxima:
    return 0
  return valor_total

Esta função calcula a aptidão de um indivíduo (solução) com base no peso e valor total dos itens selecionados.

#### Geração da População Inicial

In [ ]:
def gerar_populacao_inicial(tamanho_populacao, numero_itens):
    populacao = []
    for _ in range(tamanho_populacao):
        individuo = [random.randint(0, 1) for _ in range(numero_itens)]
        if random.random() < 0.3:
            individuo = [0] * numero_itens
            peso_atual = 0
            for i in sorted(range(numero_itens), key=lambda x: valores[x] / pesos[x], reverse=True):
                if peso_atual + pesos[i] <= capacidade_maxima:
                    individuo[i] = 1
                    peso_atual += pesos[i]
        populacao.append(individuo)
    return populacao

Aqui, geramos uma população inicial de indivíduos aleatórios.

#### Seleção


In [ ]:
def selecao(populacao, aptidoes):
    total_aptidao = sum(aptidoes)
    if total_aptidao == 0:
        return random.choice(populacao)
    pick = random.uniform(0, total_aptidao)
    current = 0
    for i, individuo in enumerate(populacao):
        current += aptidoes[i]
        if current > pick:
            return individuo

A seleção é feita com base na aptidão dos indivíduos, onde indivíduos mais aptos têm maior chance de serem escolhidos.

#### Cruzamento e Mutação

In [ ]:
def cruzamento(pai1, pai2):
    pontos_corte = sorted(random.sample(range(1, len(pai1)), 2))
    filho1 = pai1[:pontos_corte[0]] + pai2[pontos_corte[0]:pontos_corte[1]] + pai1[pontos_corte[1]:]
    filho2 = pai2[:pontos_corte[0]] + pai1[pontos_corte[0]:pontos_corte[1]] + pai2[pontos_corte[1]:]
    return filho1, filho2

def mutacao(individuo, taxa_mutacao):
    for i in range(len(individuo)):
        if random.random() < taxa_mutacao:
            individuo[i] = 1 - individuo[i]
    return individuo



### 4. Execução do Algoritmo Genético

In [ ]:
def algoritmo_genetico():
    populacao = gerar_populacao_inicial(tamanho_populacao, N_ITENS)
    best_fitness_values = []

    for geracao in range(numero_geracoes):
        aptidoes = [funcao_aptidao(individuo) for individuo in populacao]
        nova_populacao = []
        melhor_individuo = populacao[aptidoes.index(max(aptidoes))]
        nova_populacao.append(melhor_individuo)
        taxa_mutacao_geracao = taxa_mutacao * (1 - geracao / numero_geracoes)

        while len(nova_populacao) < tamanho_populacao:
            pai1 = selecao(populacao, aptidoes)
            pai2 = selecao(populacao, aptidoes)
            filho1, filho2 = cruzamento(pai1, pai2)
            nova_populacao.append(mutacao(filho1, taxa_mutacao_geracao))
            if len(nova_populacao) < tamanho_populacao:
                nova_populacao.append(mutacao(filho2, taxa_mutacao_geracao))

        populacao = nova_populacao
        best_fitness = max(aptidoes)
        best_fitness_values.append(best_fitness)
        best_individuo = populacao[aptidoes.index(best_fitness)]
        capacidade_usada = sum([pesos[i] for i in range(len(best_individuo)) if best_individuo[i] == 1])
        print(f"Geração {geracao + 1}: Melhor valor = {best_fitness}, Melhor indivíduo = {best_individuo}, Capacidade usada = {capacidade_usada}/{capacidade_maxima}")

    melhor_individuo = populacao[aptidoes.index(max(aptidoes))]
    print('Melhor solução encontrada:', melhor_individuo)
    print('Valor total:', funcao_aptidao(melhor_individuo))
    capacidade_usada = sum([pesos[i] for i in range(len(melhor_individuo)) if melhor_individuo[i] == 1])
    print(f'Capacidade total usada: {capacidade_usada}/{capacidade_maxima}')

    return melhor_individuo, funcao_aptidao(melhor_individuo), capacidade_usada

solucao_genetico, valor_total_genetico, capacidade_usada_genetico = algoritmo_genetico()

Geração 1: Melhor valor = 146, Melhor indivíduo = [1, 0, 1, 0, 0, 0], Capacidade usada = 136/190
Geração 2: Melhor valor = 146, Melhor indivíduo = [1, 1, 0, 1, 0, 0], Capacidade usada = 179/190
Geração 3: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 1, 1, 1], Capacidade usada = 271/190
Geração 4: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 0, 1, 0], Capacidade usada = 190/190
Geração 5: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 0, 1, 0], Capacidade usada = 190/190
Geração 6: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 0, 1, 0], Capacidade usada = 190/190
Geração 7: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 0, 1, 0], Capacidade usada = 190/190
Geração 8: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 0, 1, 0], Capacidade usada = 190/190
Geração 9: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 0, 1, 0], Capacidade usada = 190/190
Geração 10: Melhor valor = 150, Melhor indivíduo = [1, 1, 0, 0, 1, 0], Capacidade usada = 190/190
Geração 11: Melhor valor = 15

### 5. Execução do algoritmo guloso

In [ ]:
def algoritmo_guloso(pesos, valores, capacidade_maxima):
    numero_itens = len(pesos)
    itens = list(range(numero_itens))

    razao = [valores[i] / pesos[i] for i in itens]
    itens_ordenados = sorted(itens, key=lambda i: razao[i], reverse=True)

    capacidade_atual = 0
    valor_total = 0
    solucao = [0] * numero_itens

    for i in itens_ordenados:
        if capacidade_atual + pesos[i] <= capacidade_maxima:
            solucao[i] = 1
            capacidade_atual += pesos[i]
            valor_total += valores[i]
        else:
            continue

    print(f"Solução Gulosa: {solucao}")
    print(f"Valor Total: {valor_total}")
    print(f"Capacidade Usada: {capacidade_atual}/{capacidade_maxima}")


    return solucao, valor_total, capacidade_atual

solucao_gulosa, valor_total_gulosa, capacidade_usada_gulosa = algoritmo_guloso(pesos, valores, capacidade_maxima)


Solução Gulosa: [1, 1, 0, 1, 0, 0]
Valor Total: 146
Capacidade Usada: 179/190


### 6. Comparação do resultado

In [ ]:
from tabulate import tabulate

gap_absoluto_1 = valor_total_genetico - valor_total_gulosa
gap_percentual_1 = (gap_absoluto_1 / valor_total_genetico) * 100

gap_absoluto_2 = valor_total_gulosa - valor_total_genetico
gap_percentual_2 = (gap_absoluto_2 / valor_total_gulosa) * 100


tabela = [
    ["Algoritmo Genetico", capacidade_usada_genetico, capacidade_maxima, valor_total_genetico, gap_absoluto_1, f"{gap_percentual_1:.2f}%", solucao_genetico],
    ["Algoritmo Guloso", capacidade_usada_gulosa, capacidade_maxima, valor_total_gulosa, gap_absoluto_2, f"{gap_percentual_2:.2f}%", solucao_gulosa]
]

cabecalho = ["Conjunto", "Capacidade Usada", "Capacidade Máxima", "Valor Total", "GAP Absoluto de Valor", "GAP Percentual Valor", "Solução"]
colalign = ["left", "right", "right", "right", "right", "right", "right"]

print(tabulate(tabela, headers=cabecalho, tablefmt="grid", colalign=colalign))

+--------------------+--------------------+---------------------+---------------+-------------------------+------------------------+--------------------+
| Conjunto           |   Capacidade Usada |   Capacidade Máxima |   Valor Total |   GAP Absoluto de Valor |   GAP Percentual Valor |            Solução |
+====================+====================+=====================+===============+=========================+========================+====================+
| Algoritmo Genetico |                190 |                 190 |           150 |                       4 |                  2.67% | [1, 1, 0, 0, 1, 0] |
+--------------------+--------------------+---------------------+---------------+-------------------------+------------------------+--------------------+
| Algoritmo Guloso   |                179 |                 190 |           146 |                      -4 |                 -2.74% | [1, 1, 0, 1, 0, 0] |
+--------------------+--------------------+---------------------+-----------